# Cleaning Variables

# TO-DO:
    - more years of data where it exists ( as far back as possible )

In [2]:
import pandas as pd
import numpy as np

In [3]:
## from CCD 2015-2016 data, 

## julies l. chambers high school, (not in CCD)
# Charlotte-Mecklenburg Virtual High ,(not in CCD)

# Merancas Middle College-CPCC (not appearing in search, but in dataset)
# Military and Global Leadership (not appearing in search, but in dataset)

## olympic 
# 'East Mecklenburg High', 'Harding University High',
#        'Independence High', 'Myers Park High', 'North Mecklenburg High',
#        'South Mecklenburg High', 'West Charlotte High',
#        'West Mecklenburg High', 'Providence High', 'Butler High',
#        'Vance High', 'Hopewell High',
#        'Phillip O Berry Academy of Technology', 'Mallard Creek High',
#        'Olympic TEAM High School', 'Performance Learning Center',
#        'Olympic High - Renaissance School',
#        'Olympic High-Math Eng Tech Science',
#        'Olympic High-Leadership and Development',
#        'Olympic High - Biotech Health Pub Admin', 'Ardrey Kell High',
#        'Cato Middle College High', 'Rocky River High',
#        'William Amos Hough High', 'Garinger High',
#        'Harper Middle College High',
#        'Charlotte Engineering Early College-UNCC',
#        'Levine Middle College High', 'eLearning Academy'

In [18]:
# pd.read_excel('\\DTSC4301\\Dataprep\\NC Stats Data\\counselors\\Staff_2015.xlsx')

FileNotFoundError: [Errno 2] No such file or directory: '\\DTSC4301\\Dataprep\\NC Stats Data\\counselors\\Staff_2015.xlsx'

In [15]:
dataset = 'NC Stats Data'
variable = 'counselors'

filepath =  '../DTSC4301/Dataprep' + '/' + dataset + '/' + variable + '/'

years = [ 2015, 2016, 2017, 2018, 2019, 2020, 2021]
counselor_df =pd.DataFrame()

## for every file in counselors folder
for x in years:

    ## open file    
    ## read as data frame
    sheet = pd.read_excel(filepath + 'Staff_' + str(x) + '.xlsx' , index_col = None, skiprows = [0, 1, 2], header=0, usecols=[1, 2, 3, 4, 5, 7, 8, 10, 11, 12])

    ## remove all but counselors
    sheet = sheet[sheet['Activity Assignment Classification'] == 'Guidance']

    ## add the year as a column
    sheet.insert(0, 'year', x)
# DTSC4301\Dataprep\NC Stats Data\counselors\Staff_2015.xlsx

    ## add to a new dataframe
    counselor_df = pd.concat([counselor_df, sheet])

counselor_df.reset_index(inplace=True)
counselor_df.drop(columns='index', inplace=True)

## checking the dataframe
counselor_df.head(10)

#counselor_df.to_csv('Counselor_Info.csv')

FileNotFoundError: [Errno 2] No such file or directory: '../DTSC4301/Dataprep/NC Stats Data/counselors/Staff_2015.xlsx'

In [14]:
# print(counselor_df['Total Funds'].size)
# print(counselor_df['Total Funds'].isna().sum())
# print(counselor_df['Total Funds'].mean())
# print(counselor_df['Total Funds'].median())
# print(counselor_df['Total Funds'].std())

In [7]:
# dataset = 'NC Stats Data'
variable = 'schools'
filepath = r'C:\Users\ricke\OneDrive\Desktop\DTSC4301A\Sprint 2' + '\\' + dataset + '\\' + variable + '\\'


schools_df = pd.read_csv(filepath + 'Schools.csv', header=0, names=['school', 'agency_code', 'zip_code'], usecols=[2, 3, 6], dtype={'agency_code': 'object'})

In [12]:
variable = 'intentions'
filepath = r'C:\Users\ricke\OneDrive\Desktop\DTSC4301A\Sprint 2' + '\\' + dataset + '\\' + variable + '\\'

intentions_df = pd.DataFrame()
years = [2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021]
# intention_subtotals= ['>>> Subtotal for Public SR >>>',
#  '>>> Subtotal for Private SR >>>',
#  '>>> Subtotal for Comm College >>>',
#  '>>> Subtotal for Private JR >>>',
#  '>>> Subtotal for Trade >>>',
#  'MILITARY',
#  'EMPLOYMENT',
#  'OTHERS **']

for x in years:


    ## open file    
    ## read as data frame
    sheet = pd.read_excel(filepath + 'Intentions_' + str(x) + '.xlsx' , index_col = None, skiprows = [0, 1, 2], header=0)


    ## add the year as a column
    sheet.insert(0, 'year', x)

    ## add to a new dataframe
    intentions_df = pd.concat([intentions_df, sheet])

## remove '>>>' from each entry
intentions_df['Graduate Intention'] = intentions_df['Graduate Intention'].str.replace('>>>','')
## remove trailing/leading whitespaces
intentions_df['Graduate Intention'] = intentions_df['Graduate Intention'].str.strip()

## reset index
intentions_df.reset_index(drop=True, inplace=True, )

intentions_df.dropna(inplace=True)

#intentions_df.to_csv('Grad_Intentions.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\ricke\\OneDrive\\Desktop\\DTSC4301A\\Sprint 2\\NC Report Card Data\\intentions\\Intentions_2014.xlsx'

In [ ]:
## getting only the percentage of graduates in each category

intentions_pivot = intentions_df[['year', 'Graduate Intention', 'Percent']]
intentions_pivot = intentions_pivot.pivot(index='year', columns='Graduate Intention', values='Percent').reset_index()
intentions_pivot

Graduate Intention,year,EMPLOYMENT,MILITARY,OTHERS **,Subtotal for Comm College,Subtotal for Private JR,Subtotal for Private SR,Subtotal for Public SR,Subtotal for Trade
0,2015,8.3,2.9,2.7,36.9,0.4,9.3,38.4,1.1
1,2016,9.3,2.7,1.9,35.1,0.5,9.5,40.0,0.9
2,2017,9.7,2.6,3.1,31.3,0.2,8.9,43.4,0.7
3,2018,9.3,2.3,1.2,35.4,0.4,9.2,41.4,0.7
4,2019,14.0,3.0,1.1,32.7,0.3,8.2,39.8,1.0
5,2020,15.0,2.2,1.0,32.3,0.2,7.8,40.4,1.0
6,2021,32.9,3.0,0.9,29.6,0.2,7.2,25.6,0.6


In [9]:
dataset = 'NC Report Card Data'
variable = 'AP Classes'
filepath = r'C:\Users\ricke\OneDrive\Desktop\DTSC4301A\Sprint 2' + '\\' + dataset + '\\' + variable + '\\'

## read excel file 
AP_df = pd.read_excel(filepath + 'NC_AP'  + '.xlsx' , index_col = None, header=0, usecols=[0, 1, 2, 3, 5])

## choose only Charlotte-Mecklenburg Schools 
AP_df = AP_df[ AP_df['agency_code'].str.startswith('600') == True]

## choose only high schools
AP_df = AP_df[ AP_df['category_code'] == 'H']

## write to CSV
#AP_df.to_csv('AP Data.csv')

In [10]:
## highschool only codes
agency_codes = list(AP_df['agency_code'].drop_duplicates())

In [25]:
# data items, missing , mean, median, sd
# print(AP_df['pct_ap_pass'].size)
# print(AP_df['pct_ap_pass'].isna().sum())
# print(AP_df['pct_ap_pass'].mean())
# print(AP_df['pct_ap_pass'].median())
# print(AP_df['pct_ap_pass'].std())

In [26]:
# print(AP_df['pct_ap_participation'].size)
# print(AP_df['pct_ap_participation'].isna().sum())
# print(AP_df['pct_ap_participation'].mean())
# print(AP_df['pct_ap_participation'].median())
# print(AP_df['pct_ap_participation'].std())

In [27]:
dataset = 'NC Report Card Data'
variable = 'CTE Certifications'
filepath = r'C:\Users\ricke\OneDrive\Desktop\DTSC4301A\Sprint 2' + '\\' + dataset + '\\' + variable + '\\'

credentials_df = pd.read_excel(filepath + 'CTE_Credentials'  + '.xlsx' , index_col = None, header=0, usecols=[0, 1, 4])

credentials_df = credentials_df.loc[ credentials_df['agency_code'].isin(agency_codes), : ]

#credentials_df.to_csv('Credentials Data.csv')

In [28]:
creds_enroll_df = pd.read_excel(filepath + 'CTE_Enrollment' +'.xlsx', index_col = None, header=None, skiprows=[0], names=['year', 'agency_code', 'students_CTE_pct'], usecols=[0,1,4])
creds_enroll_df

,year,agency_code,students_CTE_pct
0,2018,00A000,6.64
1,2018,00B000,1.62
2,2018,010303,41.36
3,2018,010310,55.86
4,2018,010324,72.23
...,...,...,...
3697,2020,990LEA,59.27
3698,2020,995316,94.67
3699,2020,995324,27.66
3700,2020,995330,80.45


In [29]:
creds_merged_df = credentials_df.merge(creds_enroll_df, on=['year', 'agency_code'])

In [30]:
# print(credentials_df['num_credentials'].size)
# print(credentials_df['num_credentials'].isna().sum())
# print(credentials_df['num_credentials'].mean())
# print(credentials_df['num_credentials'].median())
# print(credentials_df['num_credentials'].std())

In [31]:
dataset = 'NC Report Card Data'
variable = 'College Enrollment'
filepath = r'C:\Users\ricke\OneDrive\Desktop\DTSC4301A\Sprint 2' + '\\' + dataset + '\\' + variable + '\\'

college_df = pd.read_excel(filepath + 'college enrollment'  + '.xlsx' , index_col = None, header=0, usecols=[1, 2, 3, 4, 6])

college_df = college_df.loc[ college_df['agency_code'].isin(agency_codes), : ]
college_df.rename( columns = {'graduation_year' : 'year'}, inplace=True)
college_df.drop(college_df[ college_df['Status'] == 'NOENRO' ].index, inplace=True)

#college_df.to_csv('College Enrollment Data.csv')

In [32]:
group_definitions = pd.read_excel(filepath + 'Subgroup_Definitions' + '.xlsx', index_col = None, header=None, names=['subgroup', 'Subgroup_Full'])
group_definitions['subgroup'] = group_definitions['subgroup'].str.strip()
## this dictionary can be used to replace all the subgroup codes with full-text 

In [33]:
college_df = college_df.merge(group_definitions, on='subgroup')

In [34]:
college_pivot = pd.pivot_table(college_df, index= ['year', 'agency_code'], columns='subgroup').reset_index()

college_pivot.columns = college_pivot.columns.to_flat_index()

college_pivot.columns = [
    'year', 
    'agency_code', 
    'enroll_All',
    'enroll_AmericanIndian',
    'enroll_Asian',
    'enroll_Black',
    'enroll_Disadvantaged',
    'enroll_EnglishLearners',
    'enroll_Female',
    'enroll_Hispanic',
    'enroll_Male', 
    'enroll_Twoormore',
    'enroll_PacificIslander',
    'enroll_Disabilities',
    'enroll_White']

In [35]:
college_pivot

,year,agency_code,enroll_All,enroll_AmericanIndian,enroll_Asian,enroll_Black,enroll_Disadvantaged,enroll_EnglishLearners,enroll_Female,enroll_Hispanic,enroll_Male,enroll_Twoormore,enroll_PacificIslander,enroll_Disabilities,enroll_White
0,2011,600302,0.8883,0.9500,0.877500,0.847000,0.8070,0.750000,0.884200,0.791600,0.892300,0.9500,NaN,0.842100,0.9094
1,2011,600334,0.8750,0.9500,0.950000,0.914200,0.8518,0.950000,0.894700,0.600000,0.833300,NaN,NaN,NaN,0.8181
2,2011,600361,0.7851,0.6666,0.809500,0.698200,0.7310,0.666600,0.809600,0.636300,0.758400,0.9000,NaN,0.407400,0.8291
3,2011,600364,0.7000,NaN,0.950000,0.625000,0.7500,NaN,0.666600,0.950000,0.750000,NaN,NaN,0.950000,NaN
4,2011,600377,0.7225,0.9500,0.888800,0.689600,0.6601,0.586200,0.745100,0.523800,0.688400,0.5714,NaN,0.300000,0.8174
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
208,2019,600567,NaN,NaN,NaN,0.709677,NaN,NaN,0.742857,0.526316,0.642857,NaN,NaN,NaN,0.7500
209,2019,600576,NaN,NaN,NaN,0.376068,NaN,NaN,0.400000,NaN,0.290323,NaN,NaN,NaN,NaN
210,2019,600579,NaN,NaN,0.521739,0.438095,NaN,NaN,0.454545,NaN,0.343195,0.5200,NaN,NaN,NaN
211,2019,600592,NaN,NaN,NaN,0.435484,NaN,0.267857,0.461538,0.247863,0.330000,NaN,NaN,0.342857,NaN


In [756]:
# EDS_enrollment = college_df[ (college_df['subgroup'] == 'EDS') & (college_df['Status'] == 'ENROLL')]

# print(EDS_enrollment['pct_enrolled'].size)
# print(EDS_enrollment['pct_enrolled'].isna().sum())
# print(EDS_enrollment['pct_enrolled'].mean())
# print(EDS_enrollment['pct_enrolled'].median())
# print(EDS_enrollment['pct_enrolled'].std())

## Merge all of it together

In [39]:
main_df = schools_df[ schools_df['agency_code'].isin(agency_codes)]

## big merge statement
main_df = main_df.merge(
    AP_df[['agency_code', 'pct_ap_pass', 'pct_ap_participation', 'year']],
    on='agency_code').merge(
        counselor_df[['year', 'Total Funds']], 
        on='year').merge(
            intentions_pivot, 
            on='year').merge(
                        college_pivot,
                        on = ['year','agency_code'],
                        how='left').merge(creds_merged_df, 
                    on=['year', 'agency_code'],
                    how='left')

In [758]:
# main_df.columns

In [759]:
main_df.rename(
    columns={
        'pct_ap_pass':'AP_pass_pct',
        'pct_ap_participation':'AP_part_pct',
        'EMPLOYMENT':'int_employ',
        'MILITARY': 'int_military', 
        'OTHERS **': 'int_other', 
        'Subtotal for Comm College':'int_commcoll', 
        'Subtotal for Private JR':'int_privjr', 
        'Subtotal for Private SR':'int_privsr',
        'Subtotal for Public SR':'int_pubsr',
        'Subtotal for Trade':'int_tradesch',
        'cred_earned_pct':'CTE_cred_pct',
        'students_CTE_pct':'CTE_enroll_pct',
        'Total Funds':'total_counselors'  }, inplace=True)

In [760]:
main_df.columns

Index(['school', 'agency_code', 'zip_code', 'AP_pass_pct', 'AP_part_pct',
       'year', 'total_counselors', 'int_employ', 'int_military', 'int_other',
       'int_commcoll', 'int_privjr', 'int_privsr', 'int_pubsr', 'int_tradesch',
       'enroll_All', 'enroll_AmericanIndian', 'enroll_Asian', 'enroll_Black',
       'enroll_Disadvantaged', 'enroll_EnglishLearners', 'enroll_Female',
       'enroll_Hispanic', 'enroll_Male', 'enroll_Twoormore',
       'enroll_PacificIslander', 'enroll_Disabilities', 'enroll_White',
       'CTE_cred_pct', 'CTE_enroll_pct'],
      dtype='object')

In [761]:
zip_codes = main_df['zip_code'].unique()
zip_codes

zip_2019_median_household_income = {
    28277: 109251, 
    28105: 117852, 
    28215: 50704, 
    28212: 46688, 
    28205: 50200, 
    28208: 42570, 
    28078: 82225, 
    28227: 43990, 
    28262: np.nan,
    28269: 60423, 
    28209: 99628, 
    28270: 82270, 
    28210: 87928,
    28214: 65706, 
    28031: 91151,
    28216: 64442, 
    28273: 67621}

In [762]:
main_df['2019_med_hh_inc'] = main_df['zip_code'].map(zip_2019_median_household_income)

year = main_df.pop('year')

main_df.insert(1, 'year', year)

median_household_income = main_df.pop('2019_med_hh_inc')

main_df.insert(4, '2019_med_hh_inc', median_household_income)

In [763]:
main_df

,school,year,agency_code,zip_code,2019_med_hh_inc,AP_pass_pct,AP_part_pct,total_counselors,int_employ,int_military,...,enroll_EnglishLearners,enroll_Female,enroll_Hispanic,enroll_Male,enroll_Twoormore,enroll_PacificIslander,enroll_Disabilities,enroll_White,CTE_cred_pct,CTE_enroll_pct
0,Ardrey Kell High School,2015,600302,28277,109251.0,0.810000,0.43000,361.0,8.3,2.9,...,NaN,0.465600,NaN,0.401600,NaN,NaN,NaN,0.545600,NaN,NaN
1,Butler High School,2015,600361,28105,117852.0,0.670000,0.23000,361.0,8.3,2.9,...,NaN,0.342282,0.076063,0.322148,NaN,NaN,NaN,0.326622,NaN,NaN
2,Cato Middle College High School,2015,600334,28215,50704.0,0.520000,0.52000,361.0,8.3,2.9,...,NaN,0.500000,0.083333,0.402778,NaN,NaN,NaN,0.305556,NaN,NaN
3,East Mecklenburg High School,2015,600377,28212,46688.0,0.390000,0.11000,361.0,8.3,2.9,...,NaN,0.373684,0.078947,0.268421,NaN,NaN,NaN,0.194737,NaN,NaN
4,Garinger High School,2015,600397,28205,50200.0,0.110000,NaN,361.0,8.3,2.9,...,0.053125,0.146875,0.059375,0.190625,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
137,Rocky River High School,2020,600457,28227,43990.0,0.221698,0.11449,508.0,15.0,2.2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,67.35
138,South Mecklenburg High School,2020,600535,28210,87928.0,0.600522,0.30399,508.0,15.0,2.2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,62.60
139,West Charlotte High School,2020,600576,28216,64442.0,0.071429,0.09406,508.0,15.0,2.2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,62.23
140,West Mecklenburg High School,2020,600579,28214,65706.0,0.117978,0.12241,508.0,15.0,2.2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,82.72


In [764]:
#main_df.to_csv('Full_Data.csv')

In [765]:
main_df.columns

Index(['school', 'year', 'agency_code', 'zip_code', '2019_med_hh_inc',
       'AP_pass_pct', 'AP_part_pct', 'total_counselors', 'int_employ',
       'int_military', 'int_other', 'int_commcoll', 'int_privjr', 'int_privsr',
       'int_pubsr', 'int_tradesch', 'enroll_All', 'enroll_AmericanIndian',
       'enroll_Asian', 'enroll_Black', 'enroll_Disadvantaged',
       'enroll_EnglishLearners', 'enroll_Female', 'enroll_Hispanic',
       'enroll_Male', 'enroll_Twoormore', 'enroll_PacificIslander',
       'enroll_Disabilities', 'enroll_White', 'CTE_cred_pct',
       'CTE_enroll_pct'],
      dtype='object')